In [16]:
import pyworld as pw
import torchaudio
import os
import json
import numpy as np
from tqdm import tqdm
import pickle
import sys
import torch

sys.path.append("../")
from config import data_path, dataset2wavpath, WORLD_SAMPLE_RATE, WORLD_FRAME_SHIFT

In [3]:
import IPython.display as display

In [14]:
def extract_world_features(
    wave_file, fs=WORLD_SAMPLE_RATE, frame_period=WORLD_FRAME_SHIFT
):
    # waveform: (1, seq)
    waveform, sample_rate = torchaudio.load(wave_file)
    waveform = torchaudio.functional.resample(
        waveform, orig_freq=sample_rate, new_freq=fs
    )
    # x: (seq)
    x = np.array(waveform[0], dtype=np.double)

    _f0, t = pw.dio(x, fs, frame_period=frame_period)  # raw pitch extractor
    f0 = pw.stonemask(x, _f0, t, fs)  # pitch refinement
    sp = pw.cheaptrick(x, f0, t, fs)  # extract smoothed spectrogram
    ap = pw.d4c(x, f0, t, fs)  # extract aperiodicity

    return f0, sp, ap, fs

def world_synthesis(f0, sp, ap, fs, frame_period=WORLD_FRAME_SHIFT):
    y = pw.synthesize(
        f0, sp, ap, fs, frame_period=frame_period
    )  # synthesize an utterance using the parameters
    return y

def save_audio(path, waveform, fs):
    waveform = torch.as_tensor(waveform, dtype=torch.int16)
    if len(waveform.size()) == 1:
        waveform = waveform[None, :]
    # print("HERE: waveform", waveform.shape, waveform.dtype, waveform)
    torchaudio.save(path, waveform, fs)

In [7]:
wave_file = '/mntnfs/lee_data1/zhangxueyao/dataset/Opencpop/segments/wavs/2001000001.wav'
display.Audio(filename=wave_file)

In [8]:
f0, sp, ap, fs = extract_world_features(wave_file)
y = world_synthesis(f0, sp, ap, fs)

In [9]:
y

array([-7.90134895e-07, -8.21721078e-07, -8.50105073e-07, ...,
        8.59462490e-06,  1.06948191e-05,  1.13606887e-05])

In [13]:
display.Audio(data=y, rate=fs)

In [33]:
waveform = torch.as_tensor(y, dtype=torch.float32)

# torchaudio.save('test.wav', waveform[None, :], fs, encoding="PCM_S", bits_per_sample=16)
torchaudio.save('test.wav', waveform[None, :], fs)